In [ ]:
import kutils
from kutils import model_helper as mh
from kutils import applications as apps
from kutils import image_utils as iu
from kutils import tensor_ops as ops
from kutils import generic as gen
import pandas as pd
from keras import backend as K

### KonCept1024 model

In [ ]:
aux_root = '/mnt/home/research/data/'
data_root = '/mnt/home/research/koniq/'
ids = pd.read_csv(data_root + 'metadata/koniq10k_distributions_sets.csv')

from keras.models import Model

with K.tf.device('/cpu:0'):                    
    # build scoring model
    base_model, preprocess_fn = apps.get_model_imagenet(apps.InceptionResNetV2)
    head = apps.fc_layers(base_model.output, name='fc', 
                          fc_sizes      = [2048, 1024, 256, 1], 
                          dropout_rates = [0.25, 0.25, 0.5, 0], 
                          batch_norm    = 2)    

    model = Model(inputs = base_model.input, outputs = head)
    print('[done]')

In [ ]:
# parameter of the generator
pre = lambda im: preprocess_fn(
                    iu.ImageAugmenter(im, remap=False).
                    crop((576,768)).
                    fliplr().result)
gen_params = dict(batch_size  = 50,
                  data_path   = data_root+'images/1024x768/',
                  process_fn  = pre, 
                  input_shape = (576,768,3),
                  outputs     = ('MOS',))

# Wrapper for the model, helps with training and testing
helper = mh.ModelHelper(model, 'KonCept1024', ids, 
                     loss='MSE', metrics=["MAE", ops.plcc_tf],
                     monitor_metric = 'val_loss', 
                     monitor_mode   = 'min',
                     multiproc   = True, workers = 8,
                     logs_root   = aux_root + 'logs/koniq',
                     models_root = aux_root + 'models/koniq',
                     gen_params  = gen_params)
helper.set_multi_gpu()
print '[done]'

In [ ]:
# check how the generator works
# a,_ = helper.test_generator()
# iu.view_stack(gen.mapmm(a[0]))

#### Train model

In [ ]:
helper.train(lr=1e-4, epochs=40)
helper.load_model()
helper.train(lr=1e-5, epochs=40)

#### Test

In [ ]:
if helper.load_model():
    apps.test_rating_model(helper, 
                           groups=10);